In [1]:
pip install flexvalue==0.4.6

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
version = '2020'
!flexvalue download-avoided-costs-data-db --version $version

In [ ]:
import os

try:
    from google.colab import files
    os.environ['DATABASE_LOCATION'] = '.'
except:
    print("This is not being run in a colab, so the download functions will not work.")
    os.environ['DATABASE_LOCATION'] = '../'
    pass
import pandas as pd
from io import BytesIO

import sqlite3

In [ ]:
database_version = '2020'

db_filepath = os.path.join(os.environ['DATABASE_LOCATION'], f"{database_version}.db")
con = sqlite3.connect(db_filepath)
df_deer_load_shapes = pd.read_sql("select * from deer_load_shapes", con=con).drop("local_pkid_", axis=1)

In [ ]:
utility_options = list(set([c.split('_')[0] for c in df_deer_load_shapes.columns if c!='hour_of_year']))
deer_load_shape_options = list(set(['_'.join(c.split('_')[1:]) for c in df_deer_load_shapes.columns if c!='hour_of_year']))

print("The following utilities are available to explore:")
print(utility_options)
print()
print("The following deer load shapes are available to explore:")
deer_load_shape_options

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.palettes import Category10_10 as palette

# Call once to configure Bokeh to display plots inline in the notebook.
output_notebook()

In [ ]:
utilities = ['PGE','SCE']
deer_load_shapes = ['RES_HVAC_EFF_AC']
column_names = [f'{utility}_{deer_load_shape}' for utility in utilities for deer_load_shape in deer_load_shapes]

p = figure(tools="xwheel_zoom,box_zoom,undo,redo,reset,save", active_scroll='xwheel_zoom', plot_width=1000, plot_height=400)
for i, column_name in enumerate(column_names):
    p.line(x=df_deer_load_shapes['hour_of_year'], y=df_deer_load_shapes[column_name], color=palette[i], legend_label=column_name, alpha=0.5)
show(p)

In [ ]:
# If you want to download the full deer load shapes file, run this cell (only works in colab)
df_deer_load_shapes.to_csv('deer_load_shape.csv')
files.download('deer_load_shape.csv')

In [ ]:
df_avoided_costs = pd.read_sql("select * from acc_electricity", con=con).drop("local_pkid_", axis=1)

In [ ]:
# If you want to download the full avoided costs file (warning, this is about 2GB), run this cell (only works in colab)
df_avoided_costs.to_csv('avoided_costs.csv')
files.download('avoided_costs.csv')